In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
#Counting how many genes/onts are in each file
for f_path in sorted(glob.glob('../../nf_pipeline/outputs/vz_Brain_20221018_direct_5_filt/gene_ont/*.csv')):
    df = pd.read_csv(f_path)
    print(f_path)
    print(df['gene'].nunique(),'genes')
    print(df['annotation'].nunique(),'onts')
    

../../nf_pipeline/outputs/vz_Brain_20221018_direct_5_filt/gene_ont/vz_Brainmap_slice1_central_gene_ont.csv
254 genes
29 onts
../../nf_pipeline/outputs/vz_Brain_20221018_direct_5_filt/gene_ont/vz_Brainmap_slice1_peripheral_gene_ont.csv
254 genes
29 onts
../../nf_pipeline/outputs/vz_Brain_20221018_direct_5_filt/gene_ont/vz_Brainmap_slice1_punctate_gene_ont.csv
254 genes
29 onts
../../nf_pipeline/outputs/vz_Brain_20221018_direct_5_filt/gene_ont/vz_Brainmap_slice1_radial_gene_ont.csv
254 genes
29 onts
../../nf_pipeline/outputs/vz_Brain_20221018_direct_5_filt/gene_ont/vz_Brainmap_slice2_central_gene_ont.csv
260 genes
42 onts
../../nf_pipeline/outputs/vz_Brain_20221018_direct_5_filt/gene_ont/vz_Brainmap_slice2_peripheral_gene_ont.csv
260 genes
42 onts
../../nf_pipeline/outputs/vz_Brain_20221018_direct_5_filt/gene_ont/vz_Brainmap_slice2_punctate_gene_ont.csv
260 genes
42 onts
../../nf_pipeline/outputs/vz_Brain_20221018_direct_5_filt/gene_ont/vz_Brainmap_slice2_radial_gene_ont.csv
260 genes
42

In [3]:
metrics = ['peripheral','central','radial','punctate']
for metric in metrics:
    print(metric)
    f_paths = glob.glob('../../nf_pipeline/outputs/vz_Brain_20221018_direct_5_filt/gene_ont/*{}*.csv'.format(metric))
    df = pd.concat((pd.read_csv(p) for p in f_paths))
    
    print('Num genes pre-filt',df['gene'].nunique())
    print('Num ontologies pre-filt',df['annotation'].nunique())
    fraction_positive = df['med_score'].gt(0).sum()/len(df)
    print('Fraction ann/gene groups with a positive score {:.3f}'.format(
        fraction_positive
    ))
    
    print('Num tot ann/gene groups',df.groupby(['annotation','gene']).ngroups)
    
    
    df_all_slices = df.groupby(['annotation','gene']).filter(lambda g: g['experiment'].nunique() >= 3)
    filt_df = df_all_slices[df_all_slices['med_gene_spots'].ge(5)]
    num_high_expression_groups = filt_df.groupby(['annotation','gene']).ngroups
    sig_df = df_all_slices[
        df_all_slices['med_gene_spots'].ge(5) &
        df_all_slices['bh_p'].le(0.05)
    ]
    
    print('Sig genes',sig_df['gene'].nunique())

    num_sig_high_expression_groups = sig_df.groupby(['annotation','gene']).ngroups
    
    
    print('Num sig ann/gene groups >=5 med_gene_spots',num_sig_high_expression_groups)
    print('Num tot ann/gene groups >=5 med_gene_spots',num_high_expression_groups)
    print('Frac sig ann/gene groups >=5 med_gene_spots',num_sig_high_expression_groups/num_high_expression_groups)
    
    num_groups_same_effect = sig_df.groupby(['annotation','gene'])['med_score'].apply(lambda v: all(v>0) or all(v<0)).sum()

    print('Number sig ann/gene groups with same direction of effect {}'.format(
        num_groups_same_effect
    ))
    
    print('Fraction sig ann/gene groups with same direction of effect {:.3f}'.format(
        num_groups_same_effect/num_high_expression_groups
    ))
    
    print('There are Blank- MERFISH genes in the significant group',sig_df['gene'].str.contains('Blank-').any())
    print('')



peripheral
Num genes pre-filt 289
Num ontologies pre-filt 42
Fraction ann/gene groups with a positive score 0.376
Num tot ann/gene groups 6151
Sig genes 158
Num sig ann/gene groups >=5 med_gene_spots 2535
Num tot ann/gene groups >=5 med_gene_spots 2877
Frac sig ann/gene groups >=5 med_gene_spots 0.881126173096976
Number sig ann/gene groups with same direction of effect 2479
Fraction sig ann/gene groups with same direction of effect 0.862
There are Blank- MERFISH genes in the significant group False

central
Num genes pre-filt 289
Num ontologies pre-filt 42
Fraction ann/gene groups with a positive score 0.632
Num tot ann/gene groups 6151
Sig genes 159
Num sig ann/gene groups >=5 med_gene_spots 2523
Num tot ann/gene groups >=5 med_gene_spots 2877
Frac sig ann/gene groups >=5 med_gene_spots 0.8769551616266945
Number sig ann/gene groups with same direction of effect 2479
Fraction sig ann/gene groups with same direction of effect 0.862
There are Blank- MERFISH genes in the significant group

In [12]:
2196/2877

0.7632950990615224